# Operazioni CRUD in MongoDB

Di seguito creeremo gli script di shell ``mongo`` per eseguire le operazioni CRUD su MongoDB e poi analizzeremo il corrispondente codice ``pymongo``.

L'installazione di ``mongo`` avviene direttamente dal [sito di MongoDB](https://www.mongodb.com/download-center/community).

L'installazione di ``pymongo`` si può effettuare tramite conda:

```bash
$ conda install -c conda-forge pymongo
```

Facciamo partire il server in locale e poi ci connettiamo con:

```bash
$ mongod --config /percorso/per/il/file/mongod.conf --fork
$ mongo
```

## Creazione di un database e inserimento documenti


```javascript
use mydb

db.inventory.insertOne(
   { item: "canvas", qty: 100, tags: ["cotton"], size: { h: 28, w: 35.5, uom: "cm" } }
)

// db.<collezione>.insertOne() inserisce un documento nella collezione indicata e se questa non esiste la crea

/*
 * Inseriamo più documenti
 */
db.inventory.insertMany([
   { item: "journal", qty: 25, tags: ["blank", "red"], size: { h: 14, w: 21, uom: "cm" } },
   { item: "mat", qty: 85, tags: ["gray"], size: { h: 27.9, w: 35.5, uom: "cm" } },
   { item: "mousepad", qty: 25, tags: ["gel", "blue"], size: { h: 19, w: 22.85, uom: "cm" } }
])
```


## Tipi di Query

```javascript
// query SELECT * FROM inventory WHERE item="canvas"
db.inventory.find( { item: "canvas" } )

// qquery SELECT * FROM inventory
db.inventory.find( { } )

// drop della collezione e ripopolamento
db.inventory.drop()

db.inventory.insertMany( [
   { item: "journal", qty: 25, size: { h: 14, w: 21, uom: "cm" }, status: "A" },
   { item: "notebook", qty: 50, size: { h: 8.5, w: 11, uom: "in" }, status: "A" },
   { item: "paper", qty: 100, size: { h: 8.5, w: 11, uom: "in" }, status: "D" },
   { item: "planner", qty: 75, size: { h: 22.85, w: 30, uom: "cm" }, status: "D" },
   { item: "postcard", qty: 45, size: { h: 10, w: 15.25, uom: "cm" }, status: "A" }
])

// selezione su un documento annidato
db.inventory.find( { size: { h: 14, w: 21, uom: "cm" } } )

// la seguente query è vuota
db.inventory.find(  { size: { w: 21, h: 14, uom: "cm" } }  )

// selezione e ordinamento con indice
db.inventory.find().sort({"qty": 1, "size.w": -1})

// query AND con uso di operatori
// l'operatore si applica in forma di documento { <campo>: {$<operatore>: valore}}
// un campo in un documento annidato si riferisce come <campo>.<campo annidato>

db.inventory.find( { "size.h": { $lt: 15 }, "size.uom": "in", status: "D" } )

// query OR - restituisce tutti i documenti
db.inventory.find( { $or: [ { qty: { $lt: 60 } }, { status: "D" } ] } )

// ALTER TABLE non esiste, si sostituiscono i singoli documenti
// poco efficiente

// replaceOne() rimpiazza il primo documento che fa il match con la query con il documento 
// che è passato come secondo argomento
db.inventory.replaceOne({ item: "journal"},{ item: "journal", status: "A", size: { h: 14, w: 21, uom: "cm" }, instock: [ { warehouse: "A", qty: 5 }, { warehouse: "C", qty: 15 } ] })
db.inventory.replaceOne({ item: "notebook"},{ item: "notebook", status: "A",  size: { h: 8.5, w: 11, uom: "in" }, instock: [ { warehouse: "C", qty: 5 } ] })
db.inventory.replaceOne({ item: "paper"},{ item: "paper", status: "D", size: { h: 8.5, w: 11, uom: "in" }, instock: [ { warehouse: "A", qty: 60 }, { warehouse: "B", qty: 15 } ] })
db.inventory.replaceOne({ item: "planner"},{ item: "planner", status: "D", size: { h: 22.85, w: 30, uom: "cm" }, instock: [ { warehouse: "A", qty: 40 }, { warehouse: "B", qty: 5 } ] })
db.inventory.replaceOne({ item: "postcard"},{ item: "postcard", status: "A", size: { h: 10, w: 15.25, uom: "cm" }, instock: [ { warehouse: "B", qty: 15 }, { warehouse: "C", qty: 35 } ] })

// query su array di documenti

// richiede che un il campo qty di *un qualunque* elemento dell'array di documenti 'instock' sia <= 20
db.inventory.find( { 'instock.qty': { $lte: 20 } } )

// richiede che un il campo qty del *primo* elemento dell'array di documenti 'instock' sia <= 20
db.inventory.find( { 'instock.0.qty': { $lte: 20 } } )

// elemMatch impone condizioni di selezione multiple sui campi di ogni documento in un array

/* il secondo argomento individua i campi da selezionare
 * 0 --> il campo non viene selezionato esplicitamente
 * 1 --> campo da selezionare (automaticamente gli altri non vengono selezionati)
 */ 
db.inventory.find( { "instock": { $elemMatch: { qty: 5, warehouse: "A" } } },{_id: 0, item:1} )

// query vuota perche' non rispetta l'ordine dei campi
db.inventory.find( { "instock": { qty: 5, warehouse: "A" } },{_id: 0, item:1} )

// query che cerca quei documenti in cui i campi *in ordine* rispettano le condizioni
db.inventory.find( { "instock": { warehouse: "A", qty: 5 } },{_id: 0, item:1} )

// visualizza solo l'ultimo elemento dell'array instock appartenente a qualunque documento che soddisfa la query
db.inventory.find( { status: "A" }, { item: 1, status: 1, instock: { $slice: -1 }, _id: 0 } )
```



## Update e Delete

```

/*
 * Aggiornamento sotto condizione 
 * tutti i record in cui "qty" è basso vengono cambiati di stato
 * l'operatore currentDate imposta la data di modifica
 * attraverso il flag lastModified che la pone alla data corrente
 *
 * db.inventory.updateOne() aggiorna solo il "primo" documento che soddisfa la condizione
 */
 
db.inventory.updateMany(
   { "instock.qty": { $lt: 20 } },
   {
     $set: { status: "P" },
     $currentDate: { lastModified: true }
   }
)

/*
 * Cancellazione con specifica del write concern di tipo "majority"
 */

db.inventory.deleteOne( { status: "A" }, {writeConcern: {w: "majority", j: 1, wtimeout: 1000}})
```


## Operazioni CRUD con pymongo

In [8]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/") # 27017 è la porta di default

db = myclient["mydb"] # anche 'db = myclient.mydb'

db.inventory.insert_one(
    {"item": "canvas",
     "qty": 100,
     "tags": ["cotton"],
     "size": {"h": 28, "w": 35.5, "uom": "cm"}})

db.inventory.insert_many([
    {"item": "journal",
     "qty": 25,
     "tags": ["blank", "red"],
     "size": {"h": 14, "w": 21, "uom": "cm"}},
    {"item": "mat",
     "qty": 85,
     "tags": ["gray"],
     "size": {"h": 27.9, "w": 35.5, "uom": "cm"}},
    {"item": "mousepad",
     "qty": 25,
     "tags": ["gel", "blue"],
     "size": {"h": 19, "w": 22.85, "uom": "cm"}}])


In [9]:
# query SELECT * FROM inventory WHERE item="canvas"
cursor = db.inventory.find({"item": "canvas"})

for doc in cursor:
    print(doc)

print('\n\n\n')

# query SELECT * FROM inventory
cursor = db.inventory.find({})

for doc in cursor:
    print(doc)

print('\n\n\n')

# query con ordinamento su indice
for doc in db.inventory.find().sort([('qty',pymongo.ASCENDING),('size.w',pymongo.DESCENDING)]):
    print(doc)

{'_id': ObjectId('5fba2f3052a1589ece0c7594'), 'item': 'canvas', 'qty': 100, 'tags': ['cotton'], 'size': {'h': 28, 'w': 35.5, 'uom': 'cm'}}




{'_id': ObjectId('5fba2f3052a1589ece0c7594'), 'item': 'canvas', 'qty': 100, 'tags': ['cotton'], 'size': {'h': 28, 'w': 35.5, 'uom': 'cm'}}
{'_id': ObjectId('5fba2f3052a1589ece0c7595'), 'item': 'journal', 'qty': 25, 'tags': ['blank', 'red'], 'size': {'h': 14, 'w': 21, 'uom': 'cm'}}
{'_id': ObjectId('5fba2f3052a1589ece0c7596'), 'item': 'mat', 'qty': 85, 'tags': ['gray'], 'size': {'h': 27.9, 'w': 35.5, 'uom': 'cm'}}
{'_id': ObjectId('5fba2f3052a1589ece0c7597'), 'item': 'mousepad', 'qty': 25, 'tags': ['gel', 'blue'], 'size': {'h': 19, 'w': 22.85, 'uom': 'cm'}}




{'_id': ObjectId('5fba2f3052a1589ece0c7597'), 'item': 'mousepad', 'qty': 25, 'tags': ['gel', 'blue'], 'size': {'h': 19, 'w': 22.85, 'uom': 'cm'}}
{'_id': ObjectId('5fba2f3052a1589ece0c7595'), 'item': 'journal', 'qty': 25, 'tags': ['blank', 'red'], 'size': {'h': 14, 'w': 21, 'uom': 'cm'}}
{

In [10]:
# drop della collezione e ripopolamento
db.inventory.drop()

In [11]:
# usiamo l'oggetto serializzato bson.son.SON al posto dei dict
# perché dobbiamo rispettare la sequenza delle chiavi

from bson.son import SON
db.inventory.insert_many([
    {"item": "journal",
     "qty": 25,
     "size": SON([("h", 14), ("w", 21), ("uom", "cm")]),
     "status": "A"},
    {"item": "notebook",
     "qty": 50,
     "size": SON([("h", 8.5), ("w", 11), ("uom", "in")]),
     "status": "A"},
    {"item": "paper",
     "qty": 100,
     "size": SON([("h", 8.5), ("w", 11), ("uom", "in")]),
     "status": "D"},
    {"item": "planner",
     "qty": 75,
     "size": SON([("h", 22.85), ("w", 30), ("uom", "cm")]),
     "status": "D"},
    {"item": "postcard",
     "qty": 45,
     "size": SON([("h", 10), ("w", 15.25), ("uom", "cm")]),
     "status": "A"}])




In [12]:
# query su un documento annidato
cursor = db.inventory.find(
    {"size": SON([("h", 14), ("w", 21), ("uom", "cm")])})

for doc in cursor:
    print(doc)

print('\n\n\n')

# query AND su documento annidato e uso di operatori
cursor = db.inventory.find(
    {"size.h": {"$lt": 15}, "size.uom": "in", "status": "D"})

for doc in cursor:
    print(doc)
    
print('\n\n\n')

# query OR
cursor = db.inventory.find(
    { "$or": [ { "qty": { "$lt": 40 } }, { "status": "A" } ] })

for doc in cursor:
    print(doc)


{'_id': ObjectId('5fba2f5352a1589ece0c7598'), 'item': 'journal', 'qty': 25, 'size': {'h': 14, 'w': 21, 'uom': 'cm'}, 'status': 'A'}




{'_id': ObjectId('5fba2f5352a1589ece0c759a'), 'item': 'paper', 'qty': 100, 'size': {'h': 8.5, 'w': 11, 'uom': 'in'}, 'status': 'D'}




{'_id': ObjectId('5fba2f5352a1589ece0c7598'), 'item': 'journal', 'qty': 25, 'size': {'h': 14, 'w': 21, 'uom': 'cm'}, 'status': 'A'}
{'_id': ObjectId('5fba2f5352a1589ece0c7599'), 'item': 'notebook', 'qty': 50, 'size': {'h': 8.5, 'w': 11, 'uom': 'in'}, 'status': 'A'}
{'_id': ObjectId('5fba2f5352a1589ece0c759c'), 'item': 'postcard', 'qty': 45, 'size': {'h': 10, 'w': 15.25, 'uom': 'cm'}, 'status': 'A'}


In [13]:
# Alter table
db.inventory.replace_one({"item": "journal"},
                         {"item": "journal",
                          "status": "A",
                          "size": {"h": 14, "w": 21, "uom": "cm"},
                          "instock": [
                              SON([("warehouse", "A"), ("qty", 5)]),
                              SON([("warehouse", "C"), ("qty", 15)])]})
db.inventory.replace_one({"item": "notebook"},
                         {"item": "notebook",
                          "status": "A",
                          "size": {"h": 8.5, "w": 11, "uom": "in"},
                          "instock": [SON([("warehouse", "C"), ("qty", 5)])]})
db.inventory.replace_one({"item": "paper"},
                         {"item": "paper",
                          "status": "D",
                          "size": {"h": 8.5, "w": 11, "uom": "in"},
                          "instock": [
                              SON([("warehouse", "A"), ("qty", 60)]),
                              SON([("warehouse", "B"), ("qty", 15)])]})
db.inventory.replace_one({"item": "planner"},
                         {"item": "planner",
                          "status": "D",
                          "size": {"h": 22.85, "w": 30, "uom": "cm"},
                          "instock": [
                              SON([("warehouse", "A"), ("qty", 40)]),
                              SON([("warehouse", "B"), ("qty", 5)])]})
db.inventory.replace_one({"item": "postcard"},
                         {"item": "postcard",
                          "status": "A",
                          "size": {"h": 10, "w": 15.25, "uom": "cm"},
                          "instock": [SON([("warehouse", "B"), ("qty", 15)]),
                                      SON([("warehouse", "C"), ("qty", 35)])]})



In [14]:
# Query su database con array di documenti annidati
cursor = db.inventory.find(
    {"instock": SON([("warehouse", "A"), ("qty", 5)])},{"_id": 0, "item": 1})

for doc in cursor:
    print(doc)
    
print('\n\n\n')

cursor = db.inventory.find(
    {"instock": SON([("qty", 5), ("warehouse", "A")])},{"_id": 0, "item": 1})

for doc in cursor:
    print(doc)
    
print('\n\n\n')

cursor = db.inventory.find(
    {"instock": {"$elemMatch": {"qty": 5, "warehouse": "A"}}},{"_id": 0, "item": 1})

for doc in cursor:
    print(doc)
    
print('\n\n\n')

cursor = db.inventory.find(
    {"status": "A"},
    {"item": 1, "status": 1, "instock": {"$slice": -1}, "_id": 0})

for doc in cursor:
    print(doc)
    


{'item': 'journal'}








{'item': 'journal'}




{'item': 'journal', 'status': 'A', 'instock': [{'warehouse': 'C', 'qty': 15}]}
{'item': 'notebook', 'status': 'A', 'instock': [{'warehouse': 'C', 'qty': 5}]}
{'item': 'postcard', 'status': 'A', 'instock': [{'warehouse': 'C', 'qty': 35}]}


In [15]:
# Update e delete

db.inventory.update_many(
    {"instock.qty": {"$lt": 20}},
    {"$set": {"status": "P"},
     "$currentDate": {"lastModified": True}})

cursor = db.inventory.find({"status": "P"},{"_id": 0})
for doc in cursor:
    print(doc)

print('\n\n\n')

db.inventory.delete_one(
    { "status": "P" }
    )

cursor = db.inventory.find({"status": "P"},{"_id": 0})
for doc in cursor:
    print(doc)



{'item': 'journal', 'status': 'P', 'size': {'h': 14, 'w': 21, 'uom': 'cm'}, 'instock': [{'warehouse': 'A', 'qty': 5}, {'warehouse': 'C', 'qty': 15}], 'lastModified': datetime.datetime(2020, 11, 22, 9, 30, 51, 924000)}
{'item': 'notebook', 'status': 'P', 'size': {'h': 8.5, 'w': 11, 'uom': 'in'}, 'instock': [{'warehouse': 'C', 'qty': 5}], 'lastModified': datetime.datetime(2020, 11, 22, 9, 30, 51, 924000)}
{'item': 'paper', 'status': 'P', 'size': {'h': 8.5, 'w': 11, 'uom': 'in'}, 'instock': [{'warehouse': 'A', 'qty': 60}, {'warehouse': 'B', 'qty': 15}], 'lastModified': datetime.datetime(2020, 11, 22, 9, 30, 51, 924000)}
{'item': 'planner', 'status': 'P', 'size': {'h': 22.85, 'w': 30, 'uom': 'cm'}, 'instock': [{'warehouse': 'A', 'qty': 40}, {'warehouse': 'B', 'qty': 5}], 'lastModified': datetime.datetime(2020, 11, 22, 9, 30, 51, 924000)}
{'item': 'postcard', 'status': 'P', 'size': {'h': 10, 'w': 15.25, 'uom': 'cm'}, 'instock': [{'warehouse': 'B', 'qty': 15}, {'warehouse': 'C', 'qty': 35}],